<a href="https://colab.research.google.com/github/Basma29/NLPTasks/blob/main/task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb

In [4]:
import pandas as pd
df = pd.read_csv("IMDB Dataset.csv", encoding="utf-8", sep=",", on_bad_lines="skip", quoting=3)

print(df.head())
print(df.columns)
print(f"Dataset has {df.shape[1]} columns")

In [ ]:
df = df.iloc[:, [0, 1]]
df.columns = ["review", "sentiment"]
df.columns = df.columns.str.strip()
df = df.dropna()
print(f"Dataset size after cleaning: {df.shape}")


Dataset size after cleaning: (33691, 2)


In [7]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
texts = df['review'].astype(str).tolist()
labels = df['sentiment'].apply(lambda x: 1 if x == "positive" else 0).values

x_train, x_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

vocab_size = 10000
max_length = 200
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(x_train)

x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq = tokenizer.texts_to_sequences(x_test)

x_train_pad = pad_sequences(x_train_seq, maxlen=max_length, padding='post', truncating='post')
x_test_pad = pad_sequences(x_test_seq, maxlen=max_length, padding='post', truncating='post')


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=32),
    LSTM(32, return_sequences=False),
    Dense(1, activation='sigmoid')
])


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.fit(x_train_pad, y_train, epochs=5, batch_size=64, validation_data=(x_test_pad, y_test))


Epoch 1/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 7s 113ms/step - accuracy: 0.9987 - loss: 0.3468 - val_accuracy: 0.9911 - val_loss: 0.0512
Epoch 2/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - accuracy: 0.9996 - loss: 0.0080 - val_accuracy: 0.9911 - val_loss: 0.0554
Epoch 3/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - accuracy: 0.9985 - loss: 0.0113 - val_accuracy: 0.9911 - val_loss: 0.0576
Epoch 4/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 98ms/step - accuracy: 0.9987 - loss: 0.0098 - val_accuracy: 0.9911 - val_loss: 0.0588
Epoch 5/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 6s 141ms/step - accuracy: 0.9996 - loss: 0.0041 - val_accuracy: 0.9911 - val_loss: 0.0602


In [10]:
test_loss, test_acc = model.evaluate(x_test_pad, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9940 - loss: 0.0411
Test Accuracy: 99.11%
